![smbcComic](https://www.smbc-comics.com/comics/1538492931-20181002.png)

In [ ]:
%matplotlib inline  
import torch
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Generating some data
In order to do this lab you're gonna need some data. Holding to the tradition of data science, let's make it up as we go. Using PyTorch's __[built in distribution sampling functions](https://pytorch.org/docs/stable/distributions.html)__ create a dataset sampled from a Bernoulli, and another dataset sampled from a normal distribution

## Sampling from a normal distribution
First, draw a sample from a normal distribution using the parameters below. Store your data in the variable named `X_norm`, and print out the sample mean and standard deviation

In [ ]:
trueMu = 1.491# mean
trueSig = 0.1876# standard deviation
nNormal = 10
#### your code here ####


**How close are the sample estimates of the mean and standard deviation to the true values? How can you improve the sample estimation of these values? What is the theorem that backs up your answer? Please describe how you might do that below and then implement your solution.** Continue to store your data in the variable `X_norm`.

Note, we care more about you explaining that you know what you're doing, rather than how close you can get your estimates. 
### Your answer here

In [ ]:
#### your code here ####

## Binary data generation
Follow the same steps as above, but instead of a normal distribution we want binary data. What distribution can you use to sample binary data? Make sure that the sample estimate probability of seeing a value of `1` is close to `trueP`. **Please plot the mean of your sample as you increase n. Say from 10 to 10,000** and confirm that the mean does converge to `trueP`. Remember to store your largest sample in the variable `X_bin`

In [ ]:
trueP = 0.23
#### your code here ####

# Part 2: Maximum Likelihood Estimation
You'll be learning more about maximum likelihood estimation (MLE) next week, but you'll have a chance to gain some intuition, and see how one can implement MLE here.

In the previous section you learned how more data can help you estimate the parameters of distributions that observations are drawn from. In the example of data drawn from a normal distribution it's intuitive how we can estimate the mean and standard deviation which parameterize the distribution which generated the data. Put another way, given the data we've seen and an idea for the function which generated this data (ie a normal distribution) we want find the parameters that likely describe how this data was generated. 

Let's examine some of the moving parts. We have some data, $X \in R^{n}$, and we know that each observation $x^{i}$ was generated with probability defined by $f(x^{i};\theta)$ where $\theta$ are the parameters that help us define this function. I know this is a little handwavy but just bear with me here. You'll get a better theoretical understanding in class. Let's look at the example of the normal distribution as a data generating function. Here $f(x^{i};\theta) = f(x^{i};\mu, \sigma) = \frac{1}{2\pi\sigma^2}exp(-\frac{(x^{i}-\mu)^2}{2\sigma^2})$ which isthe probability density function of the __[normal distribution](https://en.wikipedia.org/wiki/Normal_distribution)__. 

So that's the probability of observing one $x^i$, but what about all $n$ of those dudes? Well since we're dealing with indpendently and identically distributed data we just multiply that function over and over to get the likelihood of observing our data. That is $f(X;\mu, \sigma) = \prod_{i = 1}^{n}f(x^{i}|\mu, \sigma) = \frac{1}{2\pi\sigma^2}^{\frac{n}{2}}exp(-\frac{\sum_{i=1}^{n}(x^{i}-\mu)^2}{2\sigma^2})$

Ok, so that was alot. But essentially we're just deriving the probability of seeing all our data. The next and final step is to take the log of the final equation above. We do this because of numerical stability. When you're dealing with small numbers, like fractions, repeatedly muliplying them can lead to underflow. $$L(X;\mu, \sigma) = \frac{-n}{2}log(2\pi\sigma^2) - \frac{1}{2\pi\sigma^2}\sum_{i=1}^{n}(x^i - \mu)^2$$

During MLE our goal is to find the parameters $\theta$ which maximize the function $L(X;\theta)$. In the cell below fix $\sigma$ at the true value of `0.1876` and calculate the likelihood function for the data observed with varied $\mu$ between 1 and 2 with a step size of 0.05. Which value of $\mu$ maximizes the function?

In [ ]:
#### your code here ####

## MLE: Normal
So from above we can see that at the true values of $\sigma$ and $\mu$ the log likelihood is maximized. How can we learn what these parameters should be in order to maximize the MLE? With a little PyTorch and gradient descent we can do just that.

## Tensors, Variables and Computational graphs
Insert Wizard of Oz "Oh my" joke here. Ok, moving on. If we're going to learn the parameters which maximize the MLE, we need to know which parameters to maximize, and how to run the MLE function as well as how to perform back propagation. Let's look at each of these pieces separately. 

### Tensors
You can kind of think of Tensors and PyTorch's specialized NumPy matrices. How are they like NumPy matrices? They... well... they can hold values in a matrix (single or multidimensional) format. And how are the specialized? Unlike NumPy matrices Tensors can make use of GPU resources to significantly speed up matrix operations, which make up a bulk of what you need to do in deep learning.

### Variables (or Tensors with a gradient)
It used to be that PyTorch had a separate object called a Variable which were just like Tensors, except they're meant to hold values which will change as we learn their optimal or "true" value. In the most current version of PyTroch these two objects have been merged into the Tensor object. When you have a Tensor which needs to store data, but also needs to be updated as we learn its optimal or "true" value you create a tensor but require that the gradient be stored. You'll see this in the code below.

Tensors hold onto a few important attributes which you can access through `Tensor.grad.data` and `Tensor.data`. One holds onto the actual values of the object, while the other holds onto the gradient. As we learn more about machine learning in this class the reason why these two pieces of information are important will become clear. One important thing to know is that every time a variable is back propogated through the gradient **accumulates** and is not replaced so you have to manually zero out the gradient with `Tensorgrad.data.zero_()`.


### Computational graphs
Computational graphs are structures which store chains of multiplications, additions, and other math functions (sin, exponential, log, Hadamard product, etc) in such a way that calculating the gradient via the chain rule is simple. Again, you will learn more about why it's important for us to do this in the future, but for now all you need to know is that PyTorch creates computational graphs which allow you to easily calculate the derivative using the chain rule. 

A computational graph is created only once you run code which starts applying math functions, multiplications, and additions to Tensors. This means it's a dynamic computation graph. And once you calculate the derivative using a backward pass it's destroyed, meaning that all the information needed to calculate the derivatives is gone. There's more to this, but if you want to learn more about it I would suggest this __[video](https://www.youtube.com/watch?v=nbJ-2G2GXL0)__ and this __[site](https://towardsdatascience.com/getting-started-with-pytorch-part-1-understanding-how-automatic-differentiation-works-5008282073ec)__.

## An example 
Let's see some of this in action. I've implemented code to learn how to maximize the MLE for data drawn from a normal distribution by learning the best $\mu$ and $\sigma$ parameters. But let's face it, it's gonna take me a while to figure out what I'm doing. So there are 4 versions of code below. All but the last one has a bug in it that prevents it from doing what we want. **Identify the bug in each version, and what needs to be fixed**. Of course you can peak ahead to see what changed, but looking at each version by itself before moving on to the next will be helpful for future debugging sessions.

Tip: The third example might be kind of hard to spot because the code runs, but it never learns anything. Why might this be? What's going on with the gradient?

### MLE for normally distributed data: Version 1
#### What went wrong?

In [ ]:


# Taking our randomly generated data and wrapping it in a Variable call
# When we set requires_grad=False this means that we don't want to update these parameters
# and don't need to compute gradients with respect to these Variables. .
x = torch.autograd.Variable(X_norm, requires_grad=False)


# Instantiate the parameters to learn. Notice how requires_grad is set to true. This is because we want
# to update this parameter with the the MLE.
learnedMu = torch.rand(1, requires_grad = False)
learnedSigma = torch.rand(1, requires_grad = False)
print("Initial Mu: {}".format(learnedMu))
print("Initial Sigma: {}".format(learnedSigma))


# This says how big of a step should we take each time we update our parameters.
# there is some instability in the updates, so you might have to run a couple of times to get
# values that look like what they should be.
learningRate = 0.000001
# learnedProb.zero_()
n = X_norm.shape[0]
#Instantiate the MLE for a normal distribution here using torch functions.
MLE = -((n/2)*torch.log(2*np.pi*(learnedSigma**2)) + (1/(2*(learnedSigma**2)))*torch.sum((X_norm - learnedMu)**2))
for myIter in range(1000):
    MLE.backward()
    # Now we run over the data multiple times, and each time we calculate how far away we are from
    # the maximized MLE and update our parameters accordingly
    if myIter % 100 == 0:
        print("log likelihood: {}, learned mu = {}, learned sigma {}".format(MLE.data.numpy(),
                                                                          learnedMu.data.numpy(),
                                                                          learnedSigma.data.numpy()))
    # SGD update for our learned parameters
    learnedMu.data = learnedMu.data + learnedMu.grad.data*learningRate
    learnedSigma.data = learnedSigma.data + learnedSigma.grad.data*learningRate

### MLE for normally distributed data: Version 2
#### What went wrong?


In [ ]:
# Taking our randomly generated data and wrapping it in a Variable call
# When we set requires_grad=False this means that we don't want to update these parameters
# and don't need to compute gradients with respect to these Variables. .
x = torch.autograd.Variable(X_norm, requires_grad=False)


# Instantiate the parameters to learn. Notice how requires_grad is set to true. This is because we want
# to update this parameter with the the MLE.
learnedMu = torch.rand(1, requires_grad = True)
learnedSigma = torch.rand(1, requires_grad = True)
print("Initial Mu: {}".format(learnedMu))
print("Initial Sigma: {}".format(learnedSigma))


# This says how big of a step should we take each time we update our parameters.
# there is some instability in the updates, so you might have to run a couple of times to get
# values that look like what they should be.
learningRate = 0.000001
# learnedProb.zero_()
n = X_norm.shape[0]
#Instantiate the MLE for a normal distribution here using torch functions.
MLE = -((n/2)*torch.log(2*np.pi*(learnedSigma**2)) + (1/(2*(learnedSigma**2)))*torch.sum((X_norm - learnedMu)**2))
for myIter in range(1000):
    MLE.backward()
    # Now we run over the data multiple times, and each time we calculate how far away we are from
    # the maximized MLE and update our parameters accordingly
    if myIter % 100 == 0:
        print("log likelihood: {}, learned mu = {}, learned sigma {}".format(MLE.data.numpy(),
                                                                          learnedMu.data.numpy(),
                                                                          learnedSigma.data.numpy()))
    # SGD update for our learned parameters
    learnedMu.data = learnedMu.data + learnedMu.grad.data*learningRate
    learnedSigma.data = learnedSigma.data + learnedSigma.grad.data*learningRate

### MLE for normally distributed data: Version 3
#### What went wrong?


In [ ]:
# Taking our randomly generated data and wrapping it in a Variable call
# When we set requires_grad=False this means that we don't want to update these parameters
# and don't need to compute gradients with respect to these Variables. .
x = torch.autograd.Variable(X_norm, requires_grad=False)


# Instantiate the parameters to learn. Notice how requires_grad is set to true. This is because we want
# to update this parameter with the the MLE.
learnedMu = torch.rand(1, requires_grad = True)
learnedSigma = torch.rand(1, requires_grad = True)
print("Initial Mu: {}".format(learnedMu))
print("Initial Sigma: {}".format(learnedSigma))


# This says how big of a step should we take each time we update our parameters.
# there is some instability in the updates, so you might have to run a couple of times to get
# values that look like what they should be.
learningRate = 0.000001
# learnedProb.zero_()
n = X_norm.shape[0]
for myIter in range(1000):
    #Instantiate the MLE for a normal distribution here using torch functions.
    MLE = -((n/2)*torch.log(2*np.pi*(learnedSigma**2)) + (1/(2*(learnedSigma**2)))*torch.sum((X_norm - learnedMu)**2))
    MLE.backward()
    # Now we run over the data multiple times, and each time we calculate how far away we are from
    # the maximized MLE and update our parameters accordingly
    if myIter % 100 == 0:
        print("log likelihood: {}, learned mu = {}, learned sigma {}".format(MLE.data.numpy(),
                                                                          learnedMu.data.numpy(),
                                                                          learnedSigma.data.numpy()))
    # SGD update for our learned parameters
    learnedMu.data = learnedMu.data + learnedMu.grad.data*learningRate
    learnedSigma.data = learnedSigma.data + learnedSigma.grad.data*learningRate

### MLE for normally distributed data: Version 4


In [ ]:
# Taking our randomly generated data and wrapping it in a Variable call
# When we set requires_grad=False this means that we don't want to update these parameters
# and don't need to compute gradients with respect to these Variables. .
x = torch.autograd.Variable(X_norm, requires_grad=False)


# Instantiate the parameters to learn. Notice how requires_grad is set to true. This is because we want
# to update this parameter with the the MLE.
learnedMu = torch.rand(1, requires_grad = True)
learnedSigma = torch.rand(1, requires_grad = True)
print("Initial Mu: {}".format(learnedMu))
print("Initial Sigma: {}".format(learnedSigma))


# This says how big of a step should we take each time we update our parameters.
# there is some instability in the updates, so you might have to run a couple of times to get
# values that look like what they should be.
learningRate = 0.000001
# learnedProb.zero_()
n = X_norm.shape[0]
for myIter in range(1000):
    #Instantiate the MLE for a normal distribution here using torch functions.
    MLE = -((n/2)*torch.log(2*np.pi*(learnedSigma**2)) + (1/(2*(learnedSigma**2)))*torch.sum((X_norm - learnedMu)**2))
    MLE.backward()
    # Now we run over the data multiple times, and each time we calculate how far away we are from
    # the maximized MLE and update our parameters accordingly
    if myIter % 100 == 0:
        print("log likelihood: {}, learned mu = {}, learned sigma {}".format(MLE.data.numpy(),
                                                                          learnedMu.data.numpy(),
                                                                          learnedSigma.data.numpy()))
    # SGD update for our learned parameters
    learnedMu.data = learnedMu.data + learnedMu.grad.data*learningRate
    learnedSigma.data = learnedSigma.data + learnedSigma.grad.data*learningRate
    # Clear out the gradient information
    learnedMu.grad.data.zero_()
    learnedSigma.grad.data.zero_()

## MLE: Bernoulli
Finally, using the binary data we generated using a Bernoulli distribution let's learn the true value for $p$. Similarly to the above formulation you'll need to create a parameter to learn, as well as define the MLE for our binary data.

In [ ]:
# Define the variable p variable to learn
#### your code here ####
#### end code here ####
# Once again, running this can be a little temperamental, so if you're not getting the right answer
# run it a few more times just ot be sure
learningRate = 0.00001
for myIter in range(1000):
    # define the MLE for iid data from a Bernoulli
    #### your code here ####
    #### end code here ####
    MLE.backward()
    if myIter % 100 == 0:
        print("log likelihood: {}, learned prob = {}, update = {}".format(MLE.data.numpy(),
                                                                          learnedP.data.numpy(),
                                                                          learnedP.grad.data.numpy()))
    # Update your learned parameter with a gradient descent step (similar to the normal case)
    #### your code here ####
    #### end code here ####
    learnedP.grad.data.zero_()